# Create Zipcode Map between cities and counties

In [9]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import sets


/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: the sets module is deprecated


## Load zip code data

In [217]:
zipdf = pd.read_csv('/Users/along528/Dropbox/insight/project/datasets/zipcodes/us_postal_codes.csv')
zipdf = zipdf.fillna(-1)
zipdf = zipdf[zipdf['Postal Code']!=-1]
zipdf

,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
0,210,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
1,211,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
2,212,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
3,213,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
4,214,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
5,215,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
6,401,Pleasantville,New York,NY,-1,41.1381,-73.7847,-1
7,501,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
8,544,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
9,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227,-1


In [206]:
zipdf[zipdf['Postal Code']==63135]
zipdf[zipdf['Place Name']=='Saint Louis']



,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
21918,48880,Saint Louis,Michigan,MI,Gratiot,43.4278,-84.5952,-1
28668,63101,Saint Louis,Missouri,MO,St. Louis (city),38.6346,-90.1913,-1
28669,63102,Saint Louis,Missouri,MO,St. Louis (city),38.6352,-90.1864,-1
28670,63103,Saint Louis,Missouri,MO,St. Louis (city),38.6332,-90.2164,-1
28671,63104,Saint Louis,Missouri,MO,St. Louis (city),38.6128,-90.2185,-1
28672,63105,Saint Louis,Missouri,MO,St. Louis,38.6459,-90.3264,-1
28673,63106,Saint Louis,Missouri,MO,St. Louis (city),38.6442,-90.2082,-1
28674,63107,Saint Louis,Missouri,MO,St. Louis (city),38.6645,-90.2125,-1
28675,63108,Saint Louis,Missouri,MO,St. Louis (city),38.6445,-90.2544,-1
28676,63109,Saint Louis,Missouri,MO,St. Louis (city),38.5855,-90.2929,-1


Fix specific places

In [228]:
test = pd.DataFrame(zipdf) #.ix('Postal Code',43581)
test.ix[test['Postal Code'] == 60018,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 60176,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 61554,'Place Name'] = 'Marquette Heights'
test.ix[test['Postal Code'] == 60015,'Place Name'] = 'Riverwoods'
test.ix[test['Postal Code'] == 60118,'Place Name'] = 'East Dundee'
test.ix[test['Postal Code'] == 28445,'Place Name'] = 'Surf City'
test.ix[test['Postal Code'] == 27244,'Place Name'] = 'Elon'
test.ix[test['Postal Code'] == 28584,'Place Name'] = 'Cape Carteret'
test.ix[test['Postal Code'] == 60527,'Place Name'] = 'Burr Ridge'
test.ix[test['Postal Code'] == 61616,'Place Name'] = 'Peoria Heights'
test.ix[test['Postal Code'] == 60073,'Place Name'] = 'Round Lake Heights'
test.ix[test['Postal Code'] == 28227,'Place Name'] = 'Mint Hill'
test.ix[test['Postal Code'] == 6066,'Place Name'] = 'Vernon'
test.ix[test['Postal Code'] == 6340,'Place Name'] = 'Groton Town'
test.ix[test['Postal Code'] == 6349,'Place Name'] = 'Groton Town'
test.ix[test['Postal Code'] == 63135,'Place Name'] = 'Ferguson'
test.ix[(test['County'].str.contains('St. Louis')) & (test['State Abbreviation']=='MO'),'County'] = 'St Louis'

zipdf = test

## Load Police Survey data

In [4]:
dbname = 'police_bjs'
username = 'along528'
pswd = 'password'
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [230]:
surveydf = pd.read_sql('SELECT * FROM police_data_table',con)
#remove state, highway, and public safety departments
surveydf = surveydf[surveydf['agency'].str.contains('STATE')==False]
surveydf = surveydf[surveydf['agency'].str.contains('HIGHWAY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('HWAY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('SAFETY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('PUBLIC')==False]
#fix some city names
surveydf
surveydf.ix[surveydf['city']=='New York','city']='New York City'
surveydf.ix[surveydf['city']=='Roxbury Crossing','city']='Boston'
surveydf.ix[(surveydf['city']=='Doral') & (surveydf['state']=='FL'),'city']='Miami'
surveydf.ix[surveydf['agency']=='ST LOUIS COUNTY POLICE DEPARTMENT','agency'] = 'SAINT LOUIS COUNTY POLICE DEPARTMENT'
surveydf.ix[surveydf['agency']=='SHERIFF SAN MATEO COUNTY','agency'] = 'SAN MATEO COUNTY SHERIFF'
surveydf.ix[surveydf['agency']=='FREDRICK COUNTY SHERIFFS OFFICE','agency'] = 'FREDERICK COUNTY SHERIFFS OFFICE'
surveydf.ix[surveydf['agency']=='OFFICE OF SHERIFF GENESEE COUNTY','agency'] = 'GENESEE COUNTY SHERIFFS OFFICE'

#surveydf

Make sure I also look at counties

After some specific fixes, there are 5 cities in CT, NC and IL from the survey data that don't match at least one zipcode

In [104]:

zipmap = {}
count = 0
for index,survey_series in surveydf.iterrows():
    survey_name = survey_series['agency']
    survey_city = survey_series['city']
    survey_state = survey_series['state']
    #if 'sherif' in survey_name.lower() or 'county' in survey_name.lower(): continue
    if survey_state!='NC' and survey_state!='IL' and survey_state!='CT': continue
    
    #if survey_state!='NC': continue # and survey_state!='IL' and survey_state!='CT': continue

    testdf = zipdf[(zipdf['Place Name']==survey_city) & (zipdf['State Abbreviation']==survey_state)]
    if survey_state not in zipmap:
        #sets_map[survey_state] = sets.Set(testdf['Postal Code'].tolist())
        zipmap[survey_state] = []
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    else:
        #sets_map[survey_state].union(sets.Set(testdf['Postal Code'].tolist()))
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    if len(testdf)==0: 
        print survey_name,survey_city,survey_state
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

SAUGET POLICE DEPT Sauget IL
WILLOWBROOK POLICE DEPT Willowbrook IL
ROUND LAKE POLICE DEPARTMENT Round Lake IL
LAUREL PARK POLICE DEPARTMENT Laurel Park NC
WHISPERING PINES POLICE DEPARTMENT Whispering Pines NC
5


In [30]:
print len(zipmap['CT'])
print len(list(sets.Set(zipmap['CT'])))


166
166


Those zipcodes in NC, CT and IL that have not been matched are

In [220]:
zip_traffic_states = zipdf[(zipdf['State Abbreviation']=='NC') |\
                           (zipdf['State Abbreviation']=='CT') |\
                           (zipdf['State Abbreviation']=='IL')]
unmatched_zip_traffic_states = zip_traffic_states[zip_traffic_states['Postal Code'].isin(list(sets.Set(zipmap['CT']))+\
                                                          list(sets.Set(zipmap['IL']))+\
                                                          list(sets.Set(zipmap['NC'])))==False]
unmatched_zip_traffic_states.to_csv('unmatched_zip_ct_il_nc.csv')

The most glaring omission from the survey data appears to beNew Haven, CT.
Most of the rest seem to be small (i.e. single zipcode) towns that I've never heard of.

As for all states, 160 in total are ummatched

In [221]:

zips = []
count = 0
for index,survey_series in surveydf.iterrows():
    survey_name = survey_series['agency']
    survey_city = survey_series['city']
    survey_state = survey_series['state']
    #if 'sherif' in survey_name.lower() or 'county' in survey_name.lower(): continue
    #if survey_state!='NC' and survey_state!='IL' and survey_state!='CT': continue
    
    #if survey_state!='NC': continue # and survey_state!='IL' and survey_state!='CT': continue

    testdf = zipdf[(zipdf['Place Name']==survey_city) & (zipdf['State Abbreviation']==survey_state)]

    zips+=testdf['Postal Code'].tolist()

    if len(testdf)==0: 
        print survey_name,survey_city,survey_state
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

HOOVER POLICE DEPARTMENT Hoover AL
HOMEWOOD POLICE DEPT Homewood AL
MIDFIELD POLICE DEPT Midfield AL
VERNON POLICE DEPARTMENT Vernon CA
HOOPA TRIBAL POLICE DEPARTMENT   CA
CHERRY HILLS VILLAGE POLICE DEPT Cherry Hills Village CO
THORNTON POLICE DEPARTMENT Thornton CO
LAKEWOOD POLICE DEPARTMENT Lakewood CO
ELSMERE BUREAU OF POLICE Elsmere DE
LAUDERHILL POLICE DEPARTMENT Lauderhill FL
SOUTH MIAMI POLICE DEPARTMENT South Miami FL
MIAMI SPRINGS POLICE DEPARTMENT Miami Springs FL
SOUTH DAYTONA POLICE DEPARTMENT South Daytona FL
RIVIERA BEACH POLICE DEPARTMENT Riviera Beach FL
CORAL SPRINGS POLICE DEPARTMENT Coral Springs FL
SUNRISE POLICE DEPARTMENT Sunrise FL
PLANTATION POLICE DEPARTMENT Plantation FL
MIRAMAR POLICE DEPARTMENT Miramar FL
MARGATE POLICE DEPARTMENT Margate FL
DAVIE POLICE DEPARTMENT Davie FL
CORAL GABLES POLICE DEPT Coral Gables FL
SURFSIDE POLICE DEPARTMENT Surfside FL
COCONUT CREEK POLICE DEPARTMENT Coconut Creek FL
DAYTONA BEACH SHORES DPS Daytona Beach Shores FL
PORT ST 

In [123]:
unmatched_zip_all_states = zipdf[zipdf['Postal Code'].isin(list(sets.Set(zips)))==False]
unmatched_zip_all_states.to_csv('unmatched_zip_all_states.csv')

Had to fix some mismatches that didn't find Boston, New York, Ferguson, Miami



Now look specifically at the sheriff's departments as these should be treated differently

In [231]:
#counties_surveydf = surveydf[(surveydf['agency'].str.contains('COUNTY')) & (surveydf['agency'].str.contains('SHERIF'))]
counties_surveydf = surveydf[(surveydf['agency'].str.contains('COUNTY')) | (surveydf['agency'].str.contains('SHERIF'))]
counties_surveydf

,SURVEYID,formtype,resptype,agcytype,agency,city,state,zipcode,swnauthemp,swnftemp,...,impgangtask,imphumntask,impterrtask,impcell,ori,csllea04_id,population,lpdsampgrp,finalwt_page1,finalwt_page2on
337,189,L,full,3,NEW CASTLE COUNTY POLICE,New Castle,DE,19720,364,348,...,0,0,0,3,DE00203,10032244,402373,100,1.049730,1.059140
340,224,L,full,3,JACKSONVILLE SHERIFFS OFFICE,Jacksonville,FL,32202,1662,1629,...,0,0,0,1,FL01602,10002960,797350,100,1.049730,1.059140
430,279,L,full,3,ATHENS-CLARKE COUNTY POLICE DEPT,Athens,GA,30605,231,226,...,0,0,0,4,GA02901,10032553,113389,100,1.049730,1.059140
443,306,L,full,3,GLYNN COUNTY POLICE DEPT,Brunswick,GA,31520,114,114,...,0,0,0,5,GA06302,10032684,58360,100,1.049730,1.059140
445,310,L,full,3,HENRY COUNTY POLICE DEPARTMENT,McDonough,GA,30253,240,227,...,0,0,0,4,GA07505,,160080,100,1.049730,1.059140
446,303,L,full,3,FULTON COUNTY POLICE DEPARTMENT,Atlanta,GA,30303,169,169,...,0,0,0,4,GA06013,10032676,185732,100,1.049730,1.059140
448,296,L,full,3,DEKALB COUNTY POLICE DEPARTMENT,Decatur,GA,30032,1154,955,...,0,0,0,2,GA04402,10032638,635588,100,1.049730,1.059140
450,288,L,full,3,CLAYTON COUNTY POLICE DEPARTMENT,Jonesboro,GA,30236,438,313,...,0,0,0,3,GA03101,10032604,224327,100,1.049730,1.059140
467,1309,S,full,3,FLOYD COUNTY POLICE DEPARTMENT,Rome,GA,30161,79,76,...,0,0,0,8,GA05701,10032667,58712,1,2.207370,2.217590
474,290,L,full,3,COBB COUNTY POLICE DEPARTMENT,Marietta,GA,30060,577,565,...,0,0,0,2,GA03302,10032608,502116,100,1.049730,1.059140


And specifically those relevant for the traffic data

In [232]:
counties_traffic_surveydf = counties_surveydf[(counties_surveydf['state']=='NC') |\
                           (counties_surveydf['state']=='CT') |\
                           (counties_surveydf['state']=='IL')]


In [233]:
def get_county_from_agency(agency):
    county = ''
    if ' CO ' in agency:
        county = agency.split(' CO ').pop(0)
    elif ' PARISH ' in agency:
        county = agency.split('PARISH').pop(0)
    else:
        county = agency.split('COUNTY').pop(0)
    if 'SHERIF' in county:
        county = county.split('SHERIF').pop(0)
    county = county.strip()
    return county.title()
zips = []
count=0
for index,series in counties_traffic_surveydf.iterrows():
    name = series['agency']
    county = get_county_from_agency(name)
    city = series['city']
    state = series['state']
    testdf = zipdf[(zipdf['County']==county) & (zipdf['State Abbreviation']==state)]
    #zips+=testdf['Postal Code'].tolist()
    if len(testdf)==0: 
        print "%s,%s,%s,%s" %(name,city,county,state)
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

DEWITT COUNTY SHERIFF,Clinton,Dewitt,IL
DUPAGE COUNTY SHERIFFS OFFICE,Wheaton,Dupage,IL
MCLEAN COUNTY SHERIFFS OFFICE,Bloomington,Mclean,IL
KANNAKEE COUNTY SHERIFFS OFFICE,Kankakee,Kannakee,IL
MORGAN COUNT SHERIFFS OFFICE,Jacksonville,Morgan Count,IL
MCHENRY COUNTY SHERIFFS OFFICE,Woodstock,Mchenry,IL
WABATH CO SHERIFFS OFFICE,Mount Carmel,Wabath,IL
7


Only 7 sherifs office don't match. Let's ignore these.

And for the rest of the country?

In [234]:
zips = []
count=0
for index,series in counties_surveydf.iterrows():
    name = series['agency']
    county = get_county_from_agency(name)
    city = series['city']
    state = series['state']
    testdf = zipdf[(zipdf['County']==county) & (zipdf['State Abbreviation']==state)]
    #zips+=testdf['Postal Code'].tolist()
    if len(testdf)==0: 
        print "%s,%s,%s,%s" %(name,city,county,state)
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

JACKSONVILLE SHERIFFS OFFICE,Jacksonville,Jacksonville,FL
ATHENS-CLARKE COUNTY POLICE DEPT,Athens,Athens-Clarke,GA
DEKALB COUNTY POLICE DEPARTMENT,Decatur,Dekalb,GA
PRINCE GEORGE'S COUNTY POLICE DEPT,Landover,Prince George'S,MD
OFFICE OF THE SHERIFF,Martinez,Office Of The,CA
SOLANO OCUNTY SHERIFF,Fairfield,Solano Ocunty,CA
OFFICE OF THE SHERIFF,San Jose,Office Of The,CA
ST LUCIE COUNTY SHERIFFS OFFICE,Fort Pierce,St Lucie,FL
ST JOHNS COUNTY SHERIFFS OFFICE,Saint Augustine,St Johns,FL
HENRY COUNTY SHERIFFS OFFICE,Labelle,Henry,FL
DEKALB COUNTY SHERIFF OFFICE,Decatur,Dekalb,GA
WORHT COUNTY SHERIFFS OFFICE,Sylvester,Worht,GA
SCREUEN COUNTY SHERIFFS OFFICE,Sylvania,Screuen,GA
DEWITT COUNTY SHERIFF,Clinton,Dewitt,IL
DUPAGE COUNTY SHERIFFS OFFICE,Wheaton,Dupage,IL
MCLEAN COUNTY SHERIFFS OFFICE,Bloomington,Mclean,IL
KANNAKEE COUNTY SHERIFFS OFFICE,Kankakee,Kannakee,IL
MORGAN COUNT SHERIFFS OFFICE,Jacksonville,Morgan Count,IL
MCHENRY COUNTY SHERIFFS OFFICE,Woodstock,Mchenry,IL
WABATH CO SHERIF

There are 53 police departments missing after fixing by hand:

St Louis, MO - St Louis County

Flint, MI - Genesee County

San Mateo, CA - San Mateo County

Fredereick, MD - Frederick County

Can we match these counties with the zipcode data?

In [ ]:
zip_traffic_states